# Conector MT5 Python

In [1]:
import pandas as pd
import MetaTrader5 as mt5

In [2]:
# Creamos un diccionario con las credenciales.

creds = {
    'path':'C:\\Program Files\\Darwinex MetaTrader 5\\terminal64.exe',
    ## MetaQuotes Account
    'login': 84666107,
    'pss':'D@D7NvSa',
    'server':'MetaQuotes-Demo',

    ## Darwinex Account
    # 'login' : 4000040131,
    # 'pss':'$iC4kd$8%K0',
    # 'server':'liveUK-mt5.darwinex.com',
    
    'timeout':60000,
    'portable':False,
}

In [3]:
creds['path']

'C:\\Program Files\\Darwinex MetaTrader 5\\terminal64.exe'

In [4]:
# Conexion a MT5
if mt5.initialize(path=creds['path'], login=creds['login'], password=creds['pss'], server=creds['server'],
        timeout=creds['timeout'], portable=creds['portable']):
    print('MT5 lanzado correctamente y conectado.')
else:
    print(f'Ha habido un problema iniciando: {mt5.last_error()}')

MT5 lanzado correctamente y conectado.


# Informacion del Terminal

In [5]:
# Solicitamos la info de MT5 pero no podemos modificarla desde Python
terminal_info_dic = mt5.terminal_info()._asdict()

In [6]:
# Lo metemos en un dataframe.
terminal_info_df = pd.DataFrame(terminal_info_dic, index=['Set'])

In [7]:
# Imprimimos el datadrame.
terminal_info_df

,community_account,community_connection,connected,dlls_allowed,trade_allowed,tradeapi_disabled,email_enabled,ftp_enabled,notifications_enabled,mqid,...,codepage,ping_last,community_balance,retransmission,company,name,language,path,data_path,commondata_path
Set,False,False,True,False,True,False,False,False,False,False,...,1252,47765,0.0,0.0,Tradeslide Trading Tech Limited,Darwinex MetaTrader 5,Spanish,C:\Program Files\Darwinex MetaTrader 5,C:\Users\joalr\AppData\Roaming\MetaQuotes\Term...,C:\Users\joalr\AppData\Roaming\MetaQuotes\Term...


# Informacion de la cuenta

In [8]:
# Aqui almacenamos la informacion de la cuenta, en un diccionario.
account_info_dict = mt5.account_info()._asdict()

# Asi al momento de necesitar saber algun dato especifico lo tendremos ordenado y segmentado.
account_info_dict['profit']
account_info_dict['balance']


99813.56

# Recuperacion de datos: Symbols

In [9]:
# Para ver la cantidad de simbolos.
mt5.symbols_total()

140

In [10]:
# Para filtrar simbolos.
mt5.symbols_get() # Devolvera todos los simbolos en una lista fea de leer.
mt5.symbols_get('*EUR*') # Devolvera todos los simbolos relacionados con el ACTIVO SELECCIONADO en una lista fea de leer.
symbols = mt5.symbols_get('*EUR*') # Los metemos en una variable.

In [11]:
# Con un bucle for recorremos solo los nombres, es mucho mejor para el ojo.
for symbol in symbols:
    print(symbol.name)

EURUSD
EURAUD
EURCAD
EURCHF
EURCZK
EURDKK
EURGBP
EURHKD
EURHUF
EURJPY
EURNOK
EURNZD
EURPLN
EURRUR
EURRUB
EURSEK
EURTRY
EURZAR
XAUEUR
XAGEUR
EURMXN
EURSGD
EURILS
EURCNH


In [12]:
# Acceder a la informacion de un simbolo en concreto.
mt5.symbol_info('EURUSD')
eurusd = mt5.symbol_info('EURUSD')._asdict() # Lo metemos como un diccionario en una variable.

In [13]:
# Igual que antes podemos acceder siempre a la informacion especifica del diccionario.
eurusd['digits']

5

In [14]:
# Lo metemos PLANAMENTE en una variable.
eurjpy = mt5.symbol_info('EURJPY')

In [15]:
# De esta forma tambien podriamos acceder, atravez de la nomenclatura del punto.
eurjpy.digits

3

##### Agregar al Market Watch

In [16]:
#Agregamos un simbolo al market watch, que creo, sera necesario para poder operarlo desde python.
if (mt5.symbol_select('EURUSD',True)):
    print('El simbolo ha sido agregado.')
else:
    print(f'Error al agregar el simbolo: {mt5.last_error()}')

El simbolo ha sido agregado.


##### InfoTICK sobre un activo.

In [17]:
#Pedimos informacion sobre el ultimo tick de un activo x.
symbol_info = mt5.symbol_info_tick('EURUSD')
symbol_info.ask

1.09194

In [18]:
mt5.symbol_info_tick('EURUSD')._asdict()

{'time': 1722621093,
 'bid': 1.09192,
 'ask': 1.09194,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1722621093314,
 'flags': 1030,
 'volume_real': 0.0}

# Adquisicion de datos OHLC
 
###### Importaremos datos de calidad con formato OHLC que es como estamos acostumbrados a verlos. Veremos la importancia de trabajar con las zonas horarias correctas.

In [19]:

from datetime import datetime
import pytz

In [20]:
symbol = 'EURNOK'
timeframe = mt5.TIMEFRAME_H1

# Recuperamos datos desde una fecha determinada
date = datetime.strptime('2023-07-27 15:00:00', '%Y-%m-%d %H:%M:%S')
date

datetime.datetime(2023, 7, 27, 15, 0)

In [21]:
# NO LO USAREMOS.
# Convertir mi zona horaria a UTC
# date.astimezone(pytz.utc)
# Esto para calcular la hora que le he dado a la hora que corresponde en UTC (ES UN DOLOR DE CABEZA)

In [22]:
# Indicar que mi zona horaria ya esta en UTC
date.replace(tzinfo=pytz.utc)
date = date.replace(tzinfo=pytz.utc)
# De esta forma directamente le indicaremos la hora en UTC, es decir no tenemos que calcular los cambios de horario.

In [23]:
# Llamamos 10 datos de tiempo desde la fecha y hora establecida. (Por eso el 10, al final)
raw_ohlc_data = mt5.copy_rates_from(symbol, timeframe, date, 10)
# Lo metemos en un DF
df = pd.DataFrame(raw_ohlc_data)
# Y pasamos finalmente la columna time del DF a formato datetime, indicando que la unidad es de segundos.
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)

In [24]:
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-07-27 06:00:00,11.17320,11.17430,11.16679,11.16680,678,102,0
2023-07-27 07:00:00,11.16650,11.16810,11.16253,11.16719,1128,102,0
2023-07-27 08:00:00,11.16739,11.18670,11.16457,11.17829,2507,102,0
2023-07-27 09:00:00,11.17989,11.19951,11.17759,11.19951,3280,102,0
2023-07-27 10:00:00,11.19759,11.19945,11.14200,11.14894,1787,102,0
2023-07-27 11:00:00,11.14869,11.16669,11.13309,11.14469,2076,102,0
2023-07-27 12:00:00,11.14454,11.17724,11.13354,11.17424,2273,102,0
2023-07-27 13:00:00,11.17429,11.19354,11.15510,11.16639,2072,102,0
2023-07-27 14:00:00,11.16634,11.17171,11.11764,11.12069,2189,102,0


In [25]:
# Llamamos 10 datos de tiempo desde la ultima vela, hacia el numero de velas anterior que le digamos.
raw_ohlc_data = mt5.copy_rates_from_pos(symbol, timeframe, 1, 10)
# Lo metemos en un DF
df = pd.DataFrame(raw_ohlc_data)
# Y pasamos finalmente la columna time del DF a formato datetime, indicando que la unidad es de segundos.
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)


# Tener en cuenta que los datos van a estar en la timezone Asia/Nicosia y en DST de US.

In [26]:
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2024-08-02 07:00:00,11.80524,11.81009,11.80239,11.80864,4764,282,0
2024-08-02 08:00:00,11.80869,11.81934,11.80351,11.80699,3975,282,0
2024-08-02 09:00:00,11.80684,11.81596,11.79258,11.81359,2913,83,0
2024-08-02 10:00:00,11.81357,11.85299,11.81024,11.84203,3711,282,0
2024-08-02 11:00:00,11.83970,11.86172,11.83288,11.85892,3170,282,0
2024-08-02 12:00:00,11.85899,11.91294,11.85654,11.90754,3067,88,0
2024-08-02 13:00:00,11.90759,11.92007,11.89077,11.90998,2738,282,0
2024-08-02 14:00:00,11.90974,11.92175,11.89315,11.89883,2512,210,0
2024-08-02 15:00:00,11.89892,11.95604,11.86713,11.92464,4936,282,0


In [27]:

#Recuperar datos entre dos fechas

date_ini = datetime.strptime('2023-07-13 15:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.utc)
date_end = datetime.strptime('2023-07-17 15:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.utc)
raw_ohlc_data = mt5.copy_rates_range(symbol, timeframe, date_ini, date_end)
df = pd.DataFrame(raw_ohlc_data)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)


In [28]:
df.tail()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-07-17 11:00:00,11.24809,11.29360,11.24781,11.29130,2391,102,0
2023-07-17 12:00:00,11.29140,11.31569,11.25354,11.29189,2347,102,0
2023-07-17 13:00:00,11.29185,11.29630,11.26695,11.27417,1986,102,0
2023-07-17 14:00:00,11.27410,11.27435,11.24279,11.24324,1810,102,0
2023-07-17 15:00:00,11.24324,11.26760,11.23560,11.26760,2640,102,0


# Adquisicion de datos en TICK

In [29]:
symbol = 'EURUSD'

# Para embellezer el codigo:
formato = '%Y-%m-%d %H:%M:%S'

# Definimos la fecha de inicio y de final
date_ini = datetime.strptime('2023-07-13 15:00:00', formato).replace(tzinfo=pytz.utc)
date_end = datetime.strptime('2023-07-17 15:00:00', formato).replace(tzinfo=pytz.utc)

In [30]:
# Recogemos desde los proximos 5000 ticks desde la fecha de inicio indicada.
df = pd.DataFrame(mt5.copy_ticks_from(symbol, date_ini, 5000, mt5.COPY_TICKS_ALL))
df['time_msc'] = pd.to_datetime(df['time_msc'], unit='ms')
df.set_index('time_msc', inplace=True)

In [31]:
df['spread'] = df['ask'] - df['bid']
df.head()

,time,bid,ask,last,volume,flags,volume_real,spread
time_msc,,,,,,,,
2023-07-13 15:00:00.022,1689260400,1.11745,1.11745,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.223,1689260400,1.11746,1.11746,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.571,1689260400,1.11748,1.11748,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.667,1689260400,1.11749,1.11749,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.741,1689260400,1.11750,1.11750,0.0,0,134,0.0,0.0


In [32]:
df_2 = pd.DataFrame(mt5.copy_ticks_range(symbol, date_ini, date_end, mt5.COPY_TICKS_ALL))
df_2['time_msc'] = pd.to_datetime(df_2['time_msc'], unit='ms')
df_2.set_index('time_msc', inplace=True)

In [33]:
df_2

,time,bid,ask,last,volume,flags,volume_real
time_msc,,,,,,,
2023-07-13 15:00:00.022,1689260400,1.11745,1.11745,0.0,0,134,0.0
2023-07-13 15:00:00.223,1689260400,1.11746,1.11746,0.0,0,134,0.0
2023-07-13 15:00:00.571,1689260400,1.11748,1.11748,0.0,0,134,0.0
2023-07-13 15:00:00.667,1689260400,1.11749,1.11749,0.0,0,134,0.0
2023-07-13 15:00:00.741,1689260400,1.11750,1.11750,0.0,0,134,0.0
...,...,...,...,...,...,...,...
2023-07-17 14:59:57.576,1689605997,1.12342,1.12342,0.0,0,134,0.0
2023-07-17 14:59:59.209,1689605999,1.12341,1.12341,0.0,0,134,0.0
2023-07-17 14:59:59.229,1689605999,1.12342,1.12342,0.0,0,134,0.0


# Envio y Cierre de Ordenes

In [43]:
# Primero, definimos el símbolo en el que queremos abrir nuestras operaciones
symbol = 'USDSEK'

# Y nos aseguramos de que dicho símbolo esté añadido al MarketWatch
if mt5.symbol_select(symbol, True):
    print("Símbolo añadido correctamente")
else:
    print(f"Ha ocurrido un error añadiendo el símbolo: {mt5.last_error()}")

Símbolo añadido correctamente


In [44]:
# #Creacion del trade Request.
# request = {
#     'action': mt5.TRADE_ACTION_DEAL, #Enviamos una orden a mercado.
#     'magic': 1010101010, #Magic number es el id de mi estrategia, debe tener 10 numeros. 
#     #'order'        : para manejar ordenes pendientes... NO ES EL CASO.
#     'symbol': symbol,
#     'volume': 1.25, #Un numero entero siempre positivo.
#     #'price'        : Siendo una orden a mercado no necesitamos este valor.
#     #'stoplimit'    : Siendo una orden a mercado no necesitamos este valor.
#     'sl': 0.0, #StopLoss
#     'tp': 0.0, #TakeProfit
#     'deviation': 50,   #Slippage que estamos dispuestos a tolerar.
#     'type': mt5.ORDER_TYPE_BUY, #Tipo de orden (Compra, venta, limits...)
#     'type_filling': mt5.ORDER_FILLING_FOK, #Forma de relleno de la orden, ver mas a detalle, en este caso fill or kill.
#     'type_time': mt5.ORDER_TIME_GTC, #Si queremos cerrarla con condicion de tiempo, en este caso Got till Cancel es para tenerla abierta hasta que la cerremos nosotros.
#     'comment': "Mi primera orden desde python."
# } 
# request  

# POR ALGUNA EXTRANIA RAZON NO SE EJECUTA BIEN CON LOS COMENTARIOS Y LAS TABULACIONES.

In [51]:

ask = mt5.symbol_info_tick(symbol).ask
bid = mt5.symbol_info_tick(symbol).bid

# Creamos el trade request necesario
request = {
    'action': mt5.TRADE_ACTION_DEAL,
    'magic': 1010101010,
    'symbol': symbol,
    'volume': 1.25,
    'sl': ask - 0.1,
    'tp': bid + 0.2,
    'price': mt5.symbol_info_tick(symbol).ask,
    'deviation': 50,
    'type': mt5.ORDER_TYPE_BUY,
    'type_filling': mt5.ORDER_FILLING_FOK,
    'type_time': mt5.ORDER_TIME_GTC,
    'comment': "Primera orden desde Python"
}

In [77]:
# Y enviamos el request para poder ejecutar la orden
order = mt5.order_send(request)
print(order)

if order.retcode == mt5.TRADE_RETCODE_DONE:
    print(f"La orden se ha ejecutado correctamente: {order}")
else:
    print(f"Ha ocurrido un error al ejecutar la orden: {mt5.last_error()}")

OrderSendResult(retcode=10009, deal=2683248592, order=2708961231, volume=1.25, price=10.57543, bid=0.0, ask=0.0, comment='Request executed', request_id=246082533, retcode_external=0, request=TradeRequest(action=1, magic=1010101010, order=0, symbol='USDSEK', volume=1.25, price=10.57124, stoplimit=0.0, sl=10.47124, tp=10.769829999999999, deviation=50, type=0, type_filling=0, type_time=0, expiration=0, comment='Primera orden desde Python', position=0, position_by=0))
La orden se ha ejecutado correctamente: OrderSendResult(retcode=10009, deal=2683248592, order=2708961231, volume=1.25, price=10.57543, bid=0.0, ask=0.0, comment='Request executed', request_id=246082533, retcode_external=0, request=TradeRequest(action=1, magic=1010101010, order=0, symbol='USDSEK', volume=1.25, price=10.57124, stoplimit=0.0, sl=10.47124, tp=10.769829999999999, deviation=50, type=0, type_filling=0, type_time=0, expiration=0, comment='Primera orden desde Python', position=0, position_by=0))


# Cierre de todas las operaciones abiertas.

In [80]:
def cerrar_todas_positiones(position):

    request = {
        'action': mt5.TRADE_ACTION_DEAL,
        'position':position.ticket,
        'magic': position.magic,
        'symbol': position.symbol,
        'volume': position.volume,
        'deviation': 50,
        'type': mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        'type_filling': mt5.ORDER_FILLING_FOK,
        'type_time': mt5.ORDER_TIME_GTC,
        'price': mt5.symbol_info_tick(position.symbol).ask if position.type == 1 else mt5.symbol_info_tick(position.symbol).bid,
        'comment': "Primera orden desde Python"
    }
    
    return mt5.order_send(request)

In [81]:
# Primero saber cuantas operaciones tenemos abiertas
positions = mt5.positions_get()

for position in positions:
    cerrar_todas_positiones(position)

